In [17]:
# Imports
import keras
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import os, glob, random, sys, math, cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

%matplotlib inline

In [7]:
# Read image categories
ROOT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(ROOT_DIR, 'images')
CATEGORIES = [folder for folder in os.listdir(IMAGE_DIR) 
               if os.path.isdir(os.path.join(IMAGE_DIR, folder))]
NUM_CLASSES = len(CATEGORIES)
print(CATEGORIES)

['orchid']


In [14]:
# TODO rewrite using keras datagen

IMAGE_DIM = (512, 512,)

# Read images and labels into arrays
images = []
labels = []
for label, category in enumerate(categories):
    print("Processing {} images...".format(category))
    folder_path = os.path.join(IMAGE_DIR, category)
    # Read images in subfolder
    for image_file_path in tqdm(glob.glob(folder_path + "/*.jpg")):
        image_data = cv2.imread(image_file_path)
        # Resize to constant resolution
        
        images.append(image_data)
        labels.append(label)

print("Read {} images and {} labels.".format(len(images), len(labels)))

0it [00:00, ?it/s]

Processing orchid images...
Read 0 images and 0 labels.


In [18]:
# Split into training and test sets
train_test_split_ratio = 0.9 # Proportion in train set
x_train, y_train, x_test, y_test = train_test_split(images, labels, train_size = train_test_split_ratio)
print(x_train.shape())
print("{} training images, {} testing images".format(len(x_train, y_train)))

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
